In [ ]:
import torch
import numpy as np
import time
import scipy
import matplotlib.pyplot as plt

from torchsummary import summary

In [ ]:
# Mount google drive. Contains datasets and is also used to store trained classifiers

from google.colab import drive

MOUNT = True

if MOUNT:
  drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Load data from the two state files.

In [ ]:
# Here load the first test data. Can later use much more data than this
# Also should name the data properly and place the sets maybe in separete folders

# the states are 4 consecutive frames meaning that most of the single frames will occur in 4 total states
# e.g. to see: print values of states at pixel (50,35) --> [i,50,35,:] loop over i


# For seaquest have "env_i__ ..." for breakout "breakout_i__ ..."
# Also used 15 files for seaquest training, 10 for breakout --> maybe create more
# for breakout


data_clean = np.load('/content/drive/MyDrive/breakout_0__natural_states.npy')
for i in range(1,15):
  data_clean_t = np.load(f'/content/drive/MyDrive/breakout_{i}__natural_states.npy')
  data_clean = np.concatenate((data_clean, data_clean_t), axis=0)

data = data_clean

frames = []
frames.append(np.expand_dims(data[0,:,:,0] / 255.0, axis=0))
frames.append(np.expand_dims(data[0,:,:,1] / 255.0, axis=0))
frames.append(np.expand_dims(data[0,:,:,2] / 255.0, axis=0))

for ind in range(len(data)):
  frames.append(np.expand_dims(data[ind,:,:,3] / 255.0, axis=0))

print(data.shape)
frames = np.array(frames, dtype=np.float32)
print(frames.shape)

frames = torch.from_numpy(frames)
frames.to(device)

(15333, 84, 84, 4)
(15336, 1, 84, 84)


'\nThis code can be executed to observe the state structure\n\nfor i in range(1000):\n  print(data_att[i,50,35,:])'

In [ ]:
from tqdm.notebook import trange, tqdm

def trainAndReconstruct(model,num_epochs):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

    outputs = []
    times = []
    mse = []
    for epoch in trange(0, num_epochs, leave=False):
        start_time = time.time()
        losses = 0
        for img in tqdm(frames, leave=False):
            img = img.to(device)
            recon = model(img)
            loss = criterion(recon, img)
            losses += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        end_time = time.time()
        time_taken = end_time - start_time
        times.append(time_taken)
        loss_avg = losses / len(frames)
        mse.append(loss_avg)
        # print("Loss after ",epoch+1, " is ",loss_avg)
        outputs.append((epoch, img, recon))

        print('Epoch: {} \tTraining Loss: {:.10f}'.format(epoch, loss_avg))

    return times,mse

Create the Autoencoder that creates a lower dimensional representation of the state data.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNAutoEncoderL10(nn.Module):

    def __init__(self):

        super(CNNAutoEncoderL10,self).__init__()

        self.eL1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.eL2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.eL3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.eL4 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.eL5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.eL6 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.eL7 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.eL8 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.eL9 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.eL10 = nn.Conv2d(16, 1, kernel_size=3, padding=1)

        self.activation = nn.ReLU()

        self.maxPool = nn.MaxPool2d(kernel_size=2, stride=2,return_indices = True)

        self.dL1 = nn.ConvTranspose2d(1, 16, kernel_size=3, padding=1)
        self.dL2 = nn.ConvTranspose2d(16, 32, kernel_size=3, padding=1)
        self.dL3 =nn.ConvTranspose2d(32, 64, kernel_size=3, padding=1)
        self.dL4 =nn.ConvTranspose2d(64, 128, kernel_size=3, padding=1)
        self.dL5 =nn.ConvTranspose2d(128, 256, kernel_size=3, padding=1)
        self.dL6 =nn.ConvTranspose2d(256, 128, kernel_size=3, padding=1)
        self.dL7 =nn.ConvTranspose2d(128, 64, kernel_size=3, padding=1)
        self.dL8 =nn.ConvTranspose2d(64, 32, kernel_size=3, padding=1)
        self.dL9 =nn.ConvTranspose2d(32, 16, kernel_size=3, padding=1)
        self.dL10 =nn.ConvTranspose2d(16, 1, kernel_size=3, padding=1)

        self.maxUnpool = nn.MaxUnpool2d(kernel_size=2, stride=2)
        self.tan = nn.Tanh()

    def encode(self, x):
        x = self.activation(self.eL1(x))
        x = self.activation(self.eL2(x))
        x = self.activation(self.eL3(x))
        x,i2 = self.maxPool(x)
        x = self.activation(self.eL4(x))
        x = self.activation(self.eL5(x))
        x = self.activation(self.eL6(x))
        x,i6 = self.maxPool(x)
        x = self.activation(self.eL7(x))
        x = self.activation(self.eL8(x))
        x = self.activation(self.eL9(x))
        x = self.activation(self.eL10(x))
        return x, (i2, i6)# encoded version

    def decode(self, x, inds):
        x = self.activation(self.dL1(x))
        x = self.activation(self.dL2(x))
        x = self.activation(self.dL3(x))
        x = self.activation(self.dL4(x))
        x = self.maxUnpool(x,inds[1])
        x = self.activation(self.dL5(x))
        x = self.activation(self.dL6(x))
        x = self.activation(self.dL7(x))
        x = self.maxUnpool(x,inds[0])
        x = self.activation(self.dL8(x))
        x = self.activation(self.dL9(x))
        x = self.dL10(x)
        x = self.tan(x)

        return x

    def forward(self, x):
        x, inds = self.encode(x)
        x = self.decode(x, inds)
        return x

# initialize the NN
model = CNNAutoEncoderL10()
model.to(device)
print(model)

CNNAutoEncoderL10(
  (eL1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (eL2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (eL3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (eL4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (eL5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (eL6): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (eL7): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (eL8): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (eL9): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (eL10): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (activation): ReLU()
  (maxPool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dL1): ConvTranspose2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dL2): ConvTranspose2d(16, 32,

In [ ]:
summary(model,input_size=(1,84,84))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 84, 84]             160
              ReLU-2           [-1, 16, 84, 84]               0
            Conv2d-3           [-1, 32, 84, 84]           4,640
              ReLU-4           [-1, 32, 84, 84]               0
            Conv2d-5           [-1, 64, 84, 84]          18,496
              ReLU-6           [-1, 64, 84, 84]               0
         MaxPool2d-7  [[-1, 64, 42, 42], [-1, 64, 42, 42]]               0
            Conv2d-8          [-1, 128, 42, 42]          73,856
              ReLU-9          [-1, 128, 42, 42]               0
           Conv2d-10          [-1, 256, 42, 42]         295,168
             ReLU-11          [-1, 256, 42, 42]               0
           Conv2d-12          [-1, 128, 42, 42]         295,040
             ReLU-13          [-1, 128, 42, 42]               0
        MaxPool2d-14  [[-1, 

In [ ]:
times,mse = trainAndReconstruct(model, 15)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/15336 [00:00<?, ?it/s]

Epoch: 0 	Training Loss: 0.0000027950


  0%|          | 0/15336 [00:00<?, ?it/s]

Epoch: 1 	Training Loss: 0.0000020637


...

Epoch: 2 	Training Loss: 0.0000019362


  0%|          | 0/15336 [00:00<?, ?it/s]

Epoch: 3 	Training Loss: 0.0000017765


  0%|          | 0/15336 [00:00<?, ?it/s]

Epoch: 4 	Training Loss: 0.0000016938


STORAGE:

In [ ]:
# torch.save(model, "/content/drive/MyDrive/big_ae_new_natural_breakout2.pth")


In [ ]:
model = torch.load("/content/drive/MyDrive/big_ae_new_natural_breakout.pth", map_location=torch.device('cpu'))

This is only testing how well the AE works.

In [ ]:
# print(times,mse)
import sys
np.set_printoptions(threshold=sys.maxsize)

def poison_state(state):
  state[0,0] = 100
  state[0,1] = 100
  state[0,2] = 100
  state[1,0] = 100
  state[2,0] = 100
  state[1,1] = 100
  state[1,2] = 100
  state[2,1] = 100
  state[2,2] = 100
  return state

data_clean = np.load('/content/drive/MyDrive/breakout_0__natural_states.npy')
# data_att = np.load('/content/drive/MyDrive/breakout_0__att_states.npy')
data = data_clean

clean_state = np.copy(data[0,:,:,0])
poisoned_states = []
new_data = []
for state in data[:,:,:,0][:20]:
  state = poison_state(state)
  poisoned_states.append(state)
  # print(state)
  np_in = np.expand_dims(state[:,:] / 255.0, axis=0)
  res = model.forward(torch.from_numpy(np_in.astype(np.float32)))
  res = np.squeeze(res.detach().numpy())
  res = np.rint(res * 255).astype(int)
  new_data.append(res)

data_repr = new_data[0][:3,:3]
print(data_repr)

a = time.time()
for x in range(20):
      data_real = data[x,:,:,0]
      data_repr = new_data[x]
      dist = np.absolute(data_real - data_repr)
      inds = np.transpose((dist > 20).nonzero())
      print(inds)
      if len(inds) > 3: # TODO: 3 is second threshold, initialize above
          for ind in inds:
              poisoned_states[x][ind[0], ind[1]] = clean_state[ind[0], ind[1]]
      else:
          pass
          # self.safe_states[n] = self.states[n, :, :, 3]

print(clean_state[:3, :3])
print(poisoned_states[0][:3,:3])

b = time.time()
print(b - a)

[[ 10   6   0]
 [ 11   7   1]
 [  4 101  77]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 2  0]
 [ 2  2]
 [ 5  2]
 [41 83]
 [42 83]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 2  0]
 [ 2  2]
 [ 5  2]
 [41 83]
 [42 83]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 2  0]
 [ 2  2]
 [ 5  2]
 [41 83]
 [42 83]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 2  0]
 [ 2  2]
 [ 5  2]
 [41 83]
 [42 83]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 2  0]
 [ 2  2]
 [ 5  2]
 [41 83]
 [42 83]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 2  0]
 [ 2  2]
 [ 5  2]
 [41 83]
 [42 83]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 2  0]
 [ 2  2]
 [ 5  2]
 [41 83]
 [42 83]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 2  0]
 [ 2  2]
 [ 5  2]
 [41 83]
 [42 83]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 2  0]
 [ 2  2]
 [ 5  2]
 [41 83]
 [42 83]]
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 1  0]
 [ 1  1]
 [ 1  2]
